In [2]:
import xml

In [4]:
import xml.etree.ElementTree as ET

In [7]:
result = ET.parse("./FRCNNLemons/Annotations/1.xml")

In [13]:
root = result.getroot()

In [41]:
b = root.findall('object')
import numpy as np
bboxes = np.empty(shape=(len(b), 4))
for i, boxes in enumerate(root.iter('object')):
    bboxes[i,0] = boxes.find('bndbox/xmin').text
    bboxes[i,1] = boxes.find('bndbox/ymin').text

    bboxes[i,2] = boxes.find('bndbox/xmin').text
    bboxes[i,3] = boxes.find('bndbox/ymin').text
    name = boxes.find('name').text
    print(name)

Unripe
Ripe
Ripe


In [1]:
from utils.VOCLemonDataset import VOCLemon

In [2]:
dataset = VOCLemon(image_list_path="./FRCNNLemons/ImageSets/Main/Test.txt", 
                   image_path="./FRCNNLemons/JPEGImages", 
                   annot_path="./FRCNNLemons/Annotations")

In [3]:
import torch
import numpy as np

In [15]:
from torchvision.transforms import v2
from torchvision.transforms.functional import resize
image, bbox = dataset[2]
orig_h, orig_w = image.shape[1], image.shape[2]
print(orig_h, orig_w)
image = resize(image, size=1000)

new_h, new_w = image.shape[1], image.shape[2]
print(new_h, new_w)

3024 4032
1000 1333


In [26]:
new_bbox = dict()
n_bboxes = bbox['boxes'].shape[0]
print(n_bboxes)
new_bbox['labels'] = bbox['labels']
new_bbox['boxes'] = torch.from_numpy(np.empty(shape=(n_bboxes, 4)))
print(new_bbox['boxes'].shape)
print(bbox['boxes'].shape)
for idx, bb in enumerate(bbox['boxes']):
    print(f"Old Boxes")
    print(bb)
    new_bbox['boxes'][idx, 0::2] = new_w/orig_w * torch.from_numpy(bb[0::2])
    new_bbox['boxes'][idx, 1::2] = new_h/orig_h * torch.from_numpy(bb[1::2])
    print(f"New scaled boxes")
    print(new_bbox['boxes'][idx])


2
torch.Size([2, 4])
(2, 4)
Old Boxes
[2297.59 1602.42 3013.62 2342.77]
New scaled boxes
tensor([759.5951, 529.9008, 996.3183, 774.7255], dtype=torch.float64)
Old Boxes
[1640.65  764.73 2217.64 1341.73]
New scaled boxes
tensor([542.4074, 252.8869, 733.1632, 443.6938], dtype=torch.float64)


In [8]:
print(new_bbox['boxes'][0])
print(img_pil.size)

tensor([759.7851, 529.7683, 996.5675, 774.5318], dtype=torch.float64)
(1333, 1000)


In [9]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2()

NameError: name 'torchvision' is not defined

In [13]:
model.eval()

TypeError: Expected input images to be of floating type (in range [0, 1]), but found type torch.uint8 instead

In [7]:
img, target = dataset[2]

In [8]:
print(target['boxes'])

tensor([[759.7851, 529.7683, 996.5675, 774.5318],
        [542.5430, 252.8237, 733.3466, 443.5829]], dtype=torch.float64)


In [11]:
from PIL import Image, ImageDraw
import torchvision
img_pil = torchvision.transforms.functional.to_pil_image(img)
img1 = ImageDraw.Draw(img_pil)
x1,y1,x2,y2 = target['boxes'][0]
img1.rectangle(xy=(x1, y1, x2, y2), outline='red', width=10)
img_pil.show()

In [13]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)

In [16]:
model.eval()
results = model(img.unsqueeze(0))

In [17]:
print(results)

[{'boxes': tensor([[5.6685e+02, 2.4296e+02, 7.3365e+02, 4.3669e+02],
        [7.5605e+02, 5.2883e+02, 9.9161e+02, 7.7855e+02],
        [7.5731e+02, 5.2919e+02, 9.9326e+02, 7.7515e+02],
        [5.3436e+02, 2.3667e+02, 7.3404e+02, 5.4256e+02],
        [7.5277e+02, 5.3203e+02, 1.0018e+03, 8.4869e+02],
        [7.5657e+02, 5.3082e+02, 9.9135e+02, 7.7841e+02],
        [4.7759e+02, 6.5753e+00, 5.3650e+02, 8.5581e+01],
        [9.4266e+02, 7.4029e+02, 1.1252e+03, 8.2847e+02],
        [3.3925e-01, 1.0885e+01, 1.1233e+03, 9.9741e+02],
        [2.0095e+01, 3.3587e+01, 5.7913e+02, 9.8244e+02],
        [1.2009e+03, 1.4127e+02, 1.3313e+03, 4.4893e+02]],
       grad_fn=<StackBackward0>), 'labels': tensor([53, 55, 53, 16, 16, 37, 53, 16, 64, 64, 16]), 'scores': tensor([0.9899, 0.7103, 0.5581, 0.3604, 0.1126, 0.0950, 0.0606, 0.0603, 0.0588,
        0.0565, 0.0509], grad_fn=<IndexBackward0>)}]


In [18]:
print(img.unsqueeze(0).shape)

torch.Size([1, 3, 1000, 1333])


In [22]:
print(results[0]['labels'])
print(results[0]['scores'])

tensor([53, 55, 53, 16, 16, 37, 53, 16, 64, 64, 16])
tensor([0.9899, 0.7103, 0.5581, 0.3604, 0.1126, 0.0950, 0.0606, 0.0603, 0.0588,
        0.0565, 0.0509], grad_fn=<IndexBackward0>)


In [27]:
from PIL import Image

img_pil = Image.open('./FRCNNLemons/JPEGImages/1.JPG')
print(f"PIL image width x height = {img_pil.size}")
img_tensor = torchvision.transforms.functional.to_tensor(img_pil)
print(f"Torch tensor image height x width = {img_tensor.shape}")


PIL image width x height = (4032, 3024)
Torch tensor image height x width = torch.Size([3, 3024, 4032])
